In [2]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_gbq
import pydata_google_auth
from concurrent.futures import ThreadPoolExecutor, as_completed
from google.cloud import bigquery
from tqdm.notebook import tqdm
from dateutil import tz
from BigQueryTools import QueryTool
import re

SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]
credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    # Set auth_local_webserver to True to have a slightly more convienient
    # authorization flow. Note, this doesn't work if you're running from a
    # notebook on a remote sever, such as over SSH or with Google Colab.
    auth_local_webserver=True,
)

%load_ext google.cloud.bigquery
%env GCLOUD_PROJECT=nbcu-ds-sandbox-a-001

env: GCLOUD_PROJECT=nbcu-ds-sandbox-a-001


# How to use

### Before running
- Change start_date variable to analysis start. 
- Change end_date variable to analysis end. 
### Monthly and Quarterly Staging Tables
- Be aware that the pipeline will overwrite the table - it does not append new data to the existing table
- bq.add() adds jobs to queue. If a job with the same name is running, bq.add() will skip the file. Use bq.cancel() to drop currently running jobs.
- bq.start() runs all jobs in queue
- bq.update() updates all jobs in queue. Jobs that are finished are listed as DONE.
- bq.cancel() drops all running jobs in queue
- WAIT UNTIL ALL JOBS ARE COMPLETED BEFORE PROCEEDING
### Compiling Results
- "Compile Results" section puts together Monthly and Quarterly Results
- "Build Gold" section puts together final gold tables, aggregating users on filters 
### Ad Hoc Pulls
- Use analysis section to pull ad hoc requests

# Push Channel

In [3]:
start_date = '2022-01-01'
end_date = '2023-12-31'

In [4]:
def get_monthly_report_dates(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    monthly_reports = {}

    while start <= end:
        month_year = start.strftime("%b_%Y")
        report_start_date = start.replace(day=1)
        next_month = start.replace(day=28) + timedelta(days=4)
        report_end_date = next_month - timedelta(days=next_month.day)
        monthly_reports[month_year] = {
            "report_start_date": report_start_date.strftime("%Y-%m-%d"),
            "report_end_date": report_end_date.strftime("%Y-%m-%d")
        }
        start = next_month

    return monthly_reports

def get_quarter(p_date) -> int:
    return (p_date.month - 1) // 3 + 1

def get_quarterly_report_dates(start_date_str, end_date_str):
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

    quarter_dates = {}

    while start_date <= end_date:
        quarter = get_quarter(start_date)
        quarter_name = 'Q{}_{}'.format(quarter, start_date.year)
        quarter_start = datetime(start_date.year, (start_date.month - 1) // 3 * 3 + 1, 1)
        quarter_end = quarter_start.replace(year=quarter_start.year + 3 * quarter // 12, month=(quarter_start.month + 3) % 12, day=1) - timedelta(days=1)
        quarter_dates[quarter_name] = {
            'report_start_date': quarter_start.strftime('%Y-%m-%d'),
            'report_end_date': quarter_end.strftime('%Y-%m-%d')
        }
        start_date = quarter_end + timedelta(days=1)

    return quarter_dates

def generate_report_variants(period_dict, suffix="report"):
    return {'_'.join([period, suffix]): {'string_format': {'report': period}, 'params': params} for period, params in period_dict.items()}

In [5]:
months = get_monthly_report_dates(start_date, end_date)
quarters = get_quarterly_report_dates(start_date, end_date)
monthly_viewing_tables = generate_report_variants(months, 'viewing')
quarterly_viewing_tables = generate_report_variants(quarters, 'viewing')
month_tables = generate_report_variants(months)
quarter_tables = generate_report_variants(quarters)
bq = QueryTool()

## Monthly

In [6]:
with open('Silver_Push_Monthly.sql', 'r') as file:
    sql_file = file.read()
    bq.add(sql_file, month_tables)

Added: Jan_2022_report	 | 	Awaiting start
Added: Feb_2022_report	 | 	Awaiting start
Added: Mar_2022_report	 | 	Awaiting start
Added: Apr_2022_report	 | 	Awaiting start
Added: May_2022_report	 | 	Awaiting start
Added: Jun_2022_report	 | 	Awaiting start
Added: Jul_2022_report	 | 	Awaiting start
Added: Aug_2022_report	 | 	Awaiting start
Added: Sep_2022_report	 | 	Awaiting start
Added: Oct_2022_report	 | 	Awaiting start
Added: Nov_2022_report	 | 	Awaiting start
Added: Dec_2022_report	 | 	Awaiting start
Added: Jan_2023_report	 | 	Awaiting start
Added: Feb_2023_report	 | 	Awaiting start
Added: Mar_2023_report	 | 	Awaiting start
Added: Apr_2023_report	 | 	Awaiting start
Added: May_2023_report	 | 	Awaiting start
Added: Jun_2023_report	 | 	Awaiting start
Added: Jul_2023_report	 | 	Awaiting start
Added: Aug_2023_report	 | 	Awaiting start
Added: Sep_2023_report	 | 	Awaiting start
Added: Oct_2023_report	 | 	Awaiting start
Added: Nov_2023_report	 | 	Awaiting start
Added: Dec_2023_report	 | 	Awaitin

## Quarterly

Run quarterly after monthly when updating churn

In [7]:
with open('Silver_Push_Quarterly.sql', 'r') as file:
    sql_file = file.read()
    bq.add(sql_file, quarter_tables)

Added: Q1_2022_report	 | 	Awaiting start
Added: Q2_2022_report	 | 	Awaiting start
Added: Q3_2022_report	 | 	Awaiting start
Added: Q4_2022_report	 | 	Awaiting start
Added: Q1_2023_report	 | 	Awaiting start
Added: Q2_2023_report	 | 	Awaiting start
Added: Q3_2023_report	 | 	Awaiting start
Added: Q4_2023_report	 | 	Awaiting start


In [8]:
bq.start()

start 36a10424-bc0c-4314-b76d-6c80606d3a81
start 1b5c1c0a-5f43-42e8-8227-be8f73c86c30
start 633abec7-e1cc-4fdd-a6f6-fdecf5c65c2b
start aa8c7c90-626f-496b-9952-b93109588b74
start 457ef1fb-64a3-453f-8655-5483f3a5944d
start e61ae7d8-a051-4c2e-bf6a-0bf82ef53f4d
start 2340f925-cdd7-4076-9c15-b347e89811a8
start 35f87dbd-ddc1-455d-a3f8-496e803cd458
start 147185a1-e93b-4e23-88df-73a177da841f
start a4eada61-037b-48f1-ab7d-024c9b2a9cba
start 52d51c25-4793-4394-af1f-311915963b0d
start 6f41b754-6c85-4519-9b8b-92ca3a6ab151
start 1809ab69-ed63-40a6-8021-0249084c69e5
start 8cab1f1c-443e-41f1-b80b-f73dbf37ea83
start 2fa0f70c-6f85-44ed-9cbd-5723c776a7b9
start 18d78c94-5258-4f41-bf0e-2f5cb3bc1bec
start 3dc319f1-1bea-45ef-aba2-20659970d4cb
start 05555666-f490-4560-8d8b-fbca91b09e9d
start 1b69001a-328f-4cbc-868f-ddf2954136e6
start ab37abb3-73a9-4f81-90ff-a7bbad8c375d
start f5bb8642-fe49-41c6-9e72-100d60382ea9
start 102d8f7b-4185-4334-a8ff-1d817de967ed
start 9e833183-e2a2-46ad-84c5-7d809e652764
start d2a63

In [11]:
bq.update()

Active jobs: 


Finished jobs: 
Jan_2022_report	 | 	52d51c25-4793-4394-af1f-311915963b0d	 | 	Created: Jan 11, 2024 10:56 AM	 | 	Last Checked: Jan 11, 2024 01:27 PM	 | 	DONE
Feb_2022_report	 | 	36a10424-bc0c-4314-b76d-6c80606d3a81	 | 	Created: Jan 11, 2024 10:56 AM	 | 	Last Checked: Jan 11, 2024 01:27 PM	 | 	DONE
Mar_2022_report	 | 	1b5c1c0a-5f43-42e8-8227-be8f73c86c30	 | 	Created: Jan 11, 2024 10:56 AM	 | 	Last Checked: Jan 11, 2024 01:27 PM	 | 	DONE
Apr_2022_report	 | 	3dc319f1-1bea-45ef-aba2-20659970d4cb	 | 	Created: Jan 11, 2024 10:56 AM	 | 	Last Checked: Jan 11, 2024 01:27 PM	 | 	DONE
May_2022_report	 | 	03fc4649-60be-4638-a5ac-8fbf26a140e1	 | 	Created: Jan 11, 2024 10:56 AM	 | 	Last Checked: Jan 11, 2024 01:27 PM	 | 	DONE
Jun_2022_report	 | 	649e95d2-4334-4b72-8885-daf58456385f	 | 	Created: Jan 11, 2024 10:56 AM	 | 	Last Checked: Jan 11, 2024 01:27 PM	 | 	DONE
Jul_2022_report	 | 	8cab1f1c-443e-41f1-b80b-f73dbf37ea83	 | 	Created: Jan 11, 2024 10:56 AM	 | 	Last Checked: Jan 11, 2024

# Compile Results

In [12]:
# Merge and group monthly results
query = """
        CREATE or replace TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.SILVER_PUSH_CHANNEL_PERFORMANCE_MONTHLY` AS 
        
        SELECT *
        FROM ( 
        """ + \
        (' UNION ALL ').join([f'(select * from `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Channel_Base_{report}`)' for report in months]) + \
        """
        )
        """
        
with bigquery.Client() as client:
    client.query(query).result()

In [13]:
# Merge and group monthly results
query = """
        CREATE or replace TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.SILVER_PUSH_CHANNEL_PERFORMANCE_QUARTERLY` AS 
        
        SELECT *
        FROM ( 
        """ + \
        (' UNION ALL ').join([f'(select * from `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Channel_Base_{report}`)' for report in quarters]) + \
        """
        )
        """
        
with bigquery.Client() as client:
    client.query(query).result()

# Build Gold

In [15]:
# Merge and group monthly results

with open('Gold_Push_Performance.sql', 'r') as file:
    query = file.read()
    with bigquery.Client() as client:
        client.query(query).result()

# Analysis

In [23]:
def gen_analysis_query_func(total=False):
    def gen_analysis_query(report): 
        return f"""
                WITH CTE_1 AS
                (
                    SELECT  @report_start_date                                                     AS Report_Month 
                        {'--' if total else ''}, Account_Type
                        ,COUNT(distinct CASE WHEN cohort = 'Targeted' THEN aid END )            AS Distinct_Cohort_Size_Targeted
                        ,COUNT(distinct CASE WHEN cohort = 'Holdout' THEN aid END)              AS Distinct_Cohort_Size_Holdout

                        ,SUM(CASE WHEN cohort = 'Targeted' THEN viewer END )                    AS Total_Returns_Targeted
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN viewer END)                      AS Total_Returns_Holdout

                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Viewing_Time END )              AS Total_Usage_Targeted
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Viewing_Time END)                AS Total_Usage_Holdout

                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Repertoire_Pavo_Method END )    AS Total_Repertoire_Targeted
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Repertoire_Pavo_Method END)      AS Total_Repertoire_Holdout

                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Distinct_Viewing_Sessions END ) AS Total_Viewing_Sessions_Targeted
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Distinct_Viewing_Sessions END)   AS Total_Viewing_Sessions_Holdout

                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Lapsed_Save_Denom END )         AS Lapsed_Save_Denom_Targeted
                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Lapsed_Save_Num END)            AS Lapsed_Save_Num_Targeted
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Lapsed_Save_Denom END )          AS Lapsed_Save_Denom_Holdout
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Lapsed_Save_Num END)             AS Lapsed_Save_Num_Holdout

                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Lapsing_Save_Denom END )        AS Lapsing_Save_Denom_Targeted
                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Lapsing_Save_Num END)           AS Lapsing_Save_Num_Targeted
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Lapsing_Save_Denom END )         AS Lapsing_Save_Denom_Holdout
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Lapsing_Save_Num END)            AS Lapsing_Save_Num_Holdout

                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Free_To_Paid_Denom END )             AS Free_To_Paid_Denom_Targeted
                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Free_To_Paid_Num END)                AS Free_To_Paid_Num_Targeted
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Free_To_Paid_Denom END )              AS Free_To_Paid_Denom_Holdout
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Free_To_Paid_Num END)                 AS Free_To_Paid_Num_Holdout

                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Net_New_Upgrade_Denom END )     AS Total_Net_New_Upgrade_Denom_Targeted
                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Net_New_Upgrade_Num END)        AS Total_Net_New_Upgrade_Num_Targeted
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Net_New_Upgrade_Denom END )      AS Total_Net_New_Upgrade_Denom_Holdout
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Net_New_Upgrade_Num END)         AS Total_Net_New_Upgrade_Num_Holdout

                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Paid_Winbacks_Denom END )       AS Total_Paid_Winbacks_Denom_Targeted
                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Paid_Winbacks_Num END)          AS Total_Paid_Winbacks_Num_Targeted
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Paid_Winbacks_Denom END )        AS Total_Paid_Winbacks_Denom_Holdout
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Paid_Winbacks_Num END)           AS Total_Paid_Winbacks_Num_Holdout

                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Cancel_Save_Denom END )         AS Total_Cancel_Save_Denom_Targeted
                        ,SUM(CASE WHEN cohort = 'Targeted' THEN Cancel_Save_Num END)            AS Total_Cancel_Save_Num_Targeted
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Cancel_Save_Denom END )          AS Total_Cancel_Save_Denom_Holdout
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN Cancel_Save_Num END)             AS Total_Cancel_Save_Num_Holdout

                        ,SUM(CASE WHEN cohort = 'Targeted' THEN EOM_Paid_Churn_Denom END )      AS EOM_Paid_Churn_Denom_Targeted
                        ,SUM(CASE WHEN cohort = 'Targeted' THEN EOM_Paid_Churn_Num END)         AS EOM_Paid_Churn_Num_Targeted
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Denom END )       AS EOM_Paid_Churn_Denom_Holdout
                        ,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Num END)          AS EOM_Paid_Churn_Num_Holdout
                    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Channel_Base_{report}`
                    WHERE paying_account_flag = 'Paying'
                    GROUP BY  1 {'--' if total else ''}, 2
                ), CTE_2 AS
                (
                    SELECT  Report_Month 
                        {'--' if total else ''}, Account_Type
                        ,Distinct_Cohort_Size_Targeted                                                        AS Total_Targeted
                        ,Distinct_Cohort_Size_Holdout                                                         AS Total_Holdout

                        ,safe_divide(Total_Returns_Targeted,Distinct_Cohort_Size_Targeted)                    AS Return_Rate_Engagers
                        ,safe_divide(Total_Returns_Holdout,Distinct_Cohort_Size_Holdout)                      AS Return_Rate_Holdout

                        ,safe_divide(Total_Usage_Targeted,Distinct_Cohort_Size_Targeted)                      AS Usage_Engagers
                        ,safe_divide(Total_Usage_Holdout,Distinct_Cohort_Size_Holdout)                        AS Usage_Holdout

                        ,safe_divide(Total_Repertoire_Targeted,Distinct_Cohort_Size_Targeted)                 AS Repertoire_Engagers
                        ,safe_divide(Total_Repertoire_Holdout,Distinct_Cohort_Size_Holdout)                   AS Repertoire_Holdout

                        ,safe_divide(Total_Viewing_Sessions_Targeted,Distinct_Cohort_Size_Targeted)           AS Sessions_Engagers
                        ,safe_divide(Total_Viewing_Sessions_Holdout,Distinct_Cohort_Size_Holdout)             AS Sessions_Holdout

                        ,Lapsed_Save_Denom_Targeted
                        ,safe_divide(Lapsed_Save_Num_Targeted,Lapsed_Save_Denom_Targeted)                     AS Lapsed_Save_Rate_Engagers
                        ,safe_divide(Lapsed_Save_Num_Holdout,Lapsed_Save_Denom_Holdout)                       AS Lapsed_Save_Rate_Holdout

                        ,Lapsing_Save_Denom_Targeted
                        ,safe_divide(Lapsing_Save_Num_Targeted,Lapsing_Save_Denom_Targeted)                   AS Lapsing_Save_Rate_Engagers
                        ,safe_divide(Lapsing_Save_Num_Holdout,Lapsing_Save_Denom_Holdout)                     AS Lapsing_Save_Rate_Holdout

                        ,Free_To_Paid_Denom_Targeted
                        ,safe_divide(Free_To_Paid_Num_Targeted,Free_To_Paid_Denom_Targeted)                           AS Free_To_Paid_Rate_Engagers
                        ,safe_divide(Free_To_Paid_Num_Holdout,Free_To_Paid_Denom_Holdout)                             AS Free_To_Paid_Rate_Holdout

                        ,Total_Net_New_Upgrade_Denom_Targeted
                        ,safe_divide(Total_Net_New_Upgrade_Num_Targeted,Total_Net_New_Upgrade_Denom_Targeted) AS Net_New_Upgrade_Rate_Engagers
                        ,safe_divide(Total_Net_New_Upgrade_Num_Holdout,Total_Net_New_Upgrade_Denom_Holdout)   AS Net_New_Upgrade_Rate_Holdout

                        ,Total_Paid_Winbacks_Denom_Targeted
                        ,safe_divide(Total_Paid_Winbacks_Num_Targeted,Total_Paid_Winbacks_Denom_Targeted)     AS Paid_Winback_Rate_Engagers
                        ,safe_divide(Total_Paid_Winbacks_Num_Holdout,Total_Paid_Winbacks_Denom_Holdout)       AS Paid_Winback_Rate_Holdout

                        ,Total_Cancel_Save_Denom_Targeted
                        ,safe_divide(Total_Cancel_Save_Num_Targeted,Total_Cancel_Save_Denom_Targeted)         AS Cancel_Save_Rate_Engagers
                        ,safe_divide(Total_Cancel_Save_Num_Holdout,Total_Cancel_Save_Denom_Holdout)           AS Cancel_Save_Rate_Holdout

                        ,EOM_Paid_Churn_Denom_Targeted
                        ,safe_divide(EOM_Paid_Churn_Num_Targeted,EOM_Paid_Churn_Denom_Targeted)               AS Paid_Churn_Rate_Engagers
                        ,safe_divide(EOM_Paid_Churn_Num_Holdout,EOM_Paid_Churn_Denom_Holdout)                 AS Paid_Churn_Rate_Holdout
                    FROM CTE_1
                )
                SELECT  Report_Month
                    {'--' if total else ''}, Account_Type
                    ,Total_Targeted
                    ,Total_Holdout
                    
                    ,Return_Rate_Engagers                                                                                  AS Return_Rate_Engagers
                    ,Return_Rate_Holdout                                                                                   AS Return_Rate_Holdout
                    ,Return_Rate_Engagers - Return_Rate_Holdout                                                            AS Return_Rate_Lift_PTS
                    ,safe_divide(Return_Rate_Engagers,Return_Rate_Holdout) *100                                            AS Return_Rate_Lift_Index
                    ,(Return_Rate_Engagers - Return_Rate_Holdout) * Total_Targeted                                         AS Returns_Incrementals

                    ,Usage_Engagers                                                                                        AS Usage_Engagers
                    ,Usage_Holdout                                                                                         AS Usage_Holdout
                    ,Usage_Engagers - Usage_Holdout                                                                        AS Usage_Lift_PTS
                    ,safe_divide(Usage_Engagers,Usage_Holdout) *100                                                        AS Usage_Lift_Index
                    ,(Usage_Engagers - Usage_Holdout) * Total_Targeted                                                     AS Usage_Incrementals

                    ,Repertoire_Engagers                                                                                   AS Repertoire_Engagers
                    ,Repertoire_Holdout                                                                                    AS Repertoire_Holdout
                    ,Repertoire_Engagers - Repertoire_Holdout                                                              AS Repertoire_Lift_PTS
                    ,safe_divide(Repertoire_Engagers,Repertoire_Holdout) *100                                              AS Repertoire_Lift_Index
                    ,(Repertoire_Engagers - Repertoire_Holdout) * Total_Targeted                                           AS Repertoire_Incrementals

                    ,Sessions_Engagers                                                                                     AS Sessions_Engagers
                    ,Sessions_Holdout                                                                                      AS Sessions_Holdout
                    ,Sessions_Engagers - Sessions_Holdout                                                                  AS Sessions_Lift_PTS
                    ,safe_divide(Sessions_Engagers,Sessions_Holdout) *100                                                  AS Sessions_Lift_Index
                    ,(Sessions_Engagers - Sessions_Holdout) * Total_Targeted                                               AS Sessions_Incrementals

                    ,Lapsed_Save_Rate_Engagers                                                                             AS Lapsed_Save_Rate_Engagers
                    ,Lapsed_Save_Rate_Holdout                                                                              AS Lapsed_Save_Rate_Holdout
                    ,Lapsed_Save_Rate_Engagers - Lapsed_Save_Rate_Holdout                                                  AS Lapsed_Save_Rate_Lift_PTS
                    ,safe_divide(Lapsed_Save_Rate_Engagers,Lapsed_Save_Rate_Holdout) *100                                  AS Lapsed_Save_Rate_Lift_Index
                    ,(Lapsed_Save_Rate_Engagers - Lapsed_Save_Rate_Holdout) * Lapsed_Save_Denom_Targeted                   AS Lapsed_Save_Incrementals

                    ,Lapsing_Save_Rate_Engagers                                                                            AS Lapsing_Save_Rate_Engagers
                    ,Lapsing_Save_Rate_Holdout                                                                             AS Lapsing_Save_Rate_Holdout
                    ,Lapsing_Save_Rate_Engagers - Lapsing_Save_Rate_Holdout                                                AS Lapsing_Save_Rate_Lift_PTS
                    ,safe_divide(Lapsing_Save_Rate_Engagers,Lapsing_Save_Rate_Holdout) *100                                AS Lapsing_Save_Rate_Lift_Index
                    ,(Lapsing_Save_Rate_Engagers - Lapsing_Save_Rate_Holdout) * Lapsing_Save_Denom_Targeted                AS Lapsing_Save_Rate_Lift_Incrementals

                    ,Free_To_Paid_Rate_Engagers                                                                                 AS Free_To_Paid_Rate_Engagers
                    ,Free_To_Paid_Rate_Holdout                                                                                  AS Free_To_Paid_Rate_Holdout
                    ,Free_To_Paid_Rate_Engagers - Free_To_Paid_Rate_Holdout                                                          AS Free_To_Paid_Rate_Lift_PTS
                    ,safe_divide(Free_To_Paid_Rate_Engagers,Free_To_Paid_Rate_Holdout) *100                                          AS Free_To_Paid_Rate_Lift_Index
                    ,(Free_To_Paid_Rate_Engagers - Free_To_Paid_Rate_Holdout) * Free_To_Paid_Denom_Targeted                              AS Free_To_Paid_Incrementals

                    ,Net_New_Upgrade_Rate_Engagers                                                                         AS Net_New_Upgrade_Rate_Engagers
                    ,Net_New_Upgrade_Rate_Holdout                                                                          AS Net_New_Upgrade_Rate_Holdout
                    ,Net_New_Upgrade_Rate_Engagers - Net_New_Upgrade_Rate_Holdout                                          AS Net_New_Upgrade_Rate_Lift_PTS
                    ,safe_divide(Net_New_Upgrade_Rate_Engagers,Net_New_Upgrade_Rate_Holdout) *100                          AS Net_New_Upgrade_Rate_Lift_Index
                    ,(Net_New_Upgrade_Rate_Engagers - Net_New_Upgrade_Rate_Holdout) * Total_Net_New_Upgrade_Denom_Targeted AS Net_New_Upgrade_Incrementals

                    ,Paid_Winback_Rate_Engagers                                                                            AS Paid_Winback_Rate_Engagers
                    ,Paid_Winback_Rate_Holdout                                                                             AS Paid_Winback_Rate_Holdout
                    ,Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout                                                AS Paid_Winback_Rate_Lift_PTS
                    ,safe_divide(Paid_Winback_Rate_Engagers,Paid_Winback_Rate_Holdout) *100                                AS Paid_Winback_Rate_Lift_Index
                    ,(Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout) * Total_Paid_Winbacks_Denom_Targeted         AS Paid_Winback_Rate_Lift_Incrementals

                    ,Cancel_Save_Rate_Engagers                                                                             AS Cancel_Save_Rate_Engagers
                    ,Cancel_Save_Rate_Holdout                                                                              AS Cancel_Save_Rate_Holdout
                    ,Cancel_Save_Rate_Engagers - Cancel_Save_Rate_Holdout                                                  AS Cancel_Save_Rate_Lift_PTS
                    ,safe_divide(Cancel_Save_Rate_Engagers,Cancel_Save_Rate_Holdout) *100                                  AS Cancel_Save_Rate_Lift_Index
                    ,(Cancel_Save_Rate_Engagers - Cancel_Save_Rate_Holdout) * Total_Cancel_Save_Denom_Targeted             AS Cancel_Save_Rate_Incrementals

                    ,Paid_Churn_Rate_Engagers                                                                              AS EOM_Paid_Churn_Rate_Engagers
                    ,Paid_Churn_Rate_Holdout                                                                               AS EOM_Paid_Churn_Rate_Holdout
                    ,Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                                                    AS EOM_Paid_Churn_Rate_Lift_PTS
                    ,safe_divide(Paid_Churn_Rate_Engagers,Paid_Churn_Rate_Holdout) *100                                    AS EOM_Paid_Churn_Rate_Lift_Index
                    ,(Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted                  AS EOM_Paid_Churn_Rate_Incrementals
                FROM CTE_2
                ORDER BY 1 {'--' if total else ''},2
                """
    return gen_analysis_query

In [24]:
def gen_analysis_query_churn(report): 
	return f"""
			WITH CTE_1 AS
			(
				SELECT  @report_start_date                                                                                     AS Report_Month
					--, Account_Type
					,paid_tenure
					,SUM(CASE WHEN (cohort = 'Targeted') AND (Account_Type = 'Paying SVOD') THEN 1 END )  AS Distinct_Cohort_Size_Targeted
					,SUM(CASE WHEN (cohort = 'Holdout') AND (Account_Type = 'Paying SVOD') THEN 1 END)         AS Distinct_Cohort_Size_Holdout
					,SUM(CASE WHEN cohort = 'Targeted' THEN EOM_Paid_Churn_Denom END)                                AS EOM_Paid_Churn_Denom_Targeted
					,SUM(CASE WHEN cohort = 'Targeted' THEN EOM_Paid_Churn_Num END)                                   AS EOM_Paid_Churn_Num_Targeted
					,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Denom END)                                       AS EOM_Paid_Churn_Denom_Holdout
					,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Num END)                                          AS EOM_Paid_Churn_Num_Holdout
				FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Channel_Base_{report}`
				GROUP BY  1,2
			), CTE_2 AS
			(
				SELECT  Report_Month
					--, Account_Type
					,paid_tenure
					,Distinct_Cohort_Size_Targeted                                          AS Targeted
					,Distinct_Cohort_Size_Holdout                                           AS Holdout
					,EOM_Paid_Churn_Denom_Targeted
					,safe_divide(EOM_Paid_Churn_Num_Targeted,EOM_Paid_Churn_Denom_Targeted) AS Paid_Churn_Rate_Engagers
					,safe_divide(EOM_Paid_Churn_Num_Holdout,EOM_Paid_Churn_Denom_Holdout)   AS Paid_Churn_Rate_Holdout
				FROM CTE_1
			)
			SELECT  Report_Month
				--, Account_Type
				,paid_tenure
				,Targeted
				,Holdout
				,Paid_Churn_Rate_Engagers                                                             AS EOM_Paid_Churn_Rate_Engagers
				,Paid_Churn_Rate_Holdout                                                              AS EOM_Paid_Churn_Rate_Holdout
				,Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                                   AS EOM_Paid_Churn_Rate_Lift_PTS
				,safe_divide(Paid_Churn_Rate_Engagers,Paid_Churn_Rate_Holdout) *100                   AS EOM_Paid_Churn_Rate_Lift_Index
				,(Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted AS EOM_Paid_Churn_Rate_Incrementals
			FROM CTE_2
			ORDER BY 1, 2
			"""

In [25]:
analyze_queries = [
    gen_analysis_query_func(total=True),
    gen_analysis_query_func(total=False),
    gen_analysis_query_churn
]

In [26]:
def build_report(report, dates, queries):
    queries = [fn(report) for fn in queries]
    dfs = []
    for q in queries:
        query_config = {
            'query': {
                'parameterMode': 'NAMED',
                'queryParameters': [
                    {
                        'name': 'report_start_date',
                        'parameterType': {'type': 'STRING'},
                        'parameterValue': {'value': dates['report_start_date']}
                    },
                    {
                        'name': 'report_end_date',
                        'parameterType': {'type': 'STRING'},
                        'parameterValue': {'value': dates['report_end_date']}
                    }
                ]
            }
        }
        dfs.append(pandas_gbq.read_gbq(q, configuration=query_config))
    return dfs

## Monthly

In [27]:
threads = []
results = []

with ThreadPoolExecutor(len(months)) as executor:
    for report, dates in months.items():
        threads.append(executor.submit(build_report, report, dates, analyze_queries))
    for future in as_completed(threads):
        results.append(future.result())

Downloading: |          |
Downloading: |          |









Downloading: |          |
Downloading: |          |




Downloading: |          |



Downloading: |          |

Downloading: |          |


Downloading: |          |
Downloading: |          |
Downloading: |          |






Downloading: |          |
Downloading: |          |



Downloading: |          |




Downloading: |          |

Downloading: |          |
Downloading: |          |


Downloading: |          |






Downloading: |          |

Downloading: |          |
Downloading: |          |


Downloading: |          |
Downloading: |          |
Downloading: |          |

Downloading: |          |






Downloading: |          |



Downloading: |          |


Downloading: |          |

Downloading: |          |
Downloading: 100%|██████████|
Downloading: |          |
Downloading:   0%|          |
Downloading: 100%|██████████|
Downloading:   0%|          |









Downloading: 100%|██████████|


Downloading: 100%|██████████

In [28]:
aggregate_reports = pd.concat([results[i][0] for i in range(len(results))]).sort_values(by='Report_Month')
acc_type_reports = pd.concat([results[i][1] for i in range(len(results))]).sort_values(by=['Report_Month', 'Account_Type'])

with pd.ExcelWriter(f'push_channel_report_monthly.xlsx') as writer:
    aggregate_reports.to_excel(writer, sheet_name='total')
    acc_type_reports.to_excel(writer, sheet_name='acc_type')

## Quarterly

In [19]:
threads = []
results = []

with ThreadPoolExecutor(len(quarters)) as executor:
    for report, dates in quarters.items():
        threads.append(executor.submit(build_report, report, dates, analyze_queries))
    for future in as_completed(threads):
        results.append(future.result())

Downloading: |          |
Downloading: |          |
Downloading: |          |
Downloading: |          |
Downloading: |          |
Downloading: |          |
Downloading:   0%|          |

Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading:   0%|          |
Downloading: 100%|██████████|
Downloading: 100%|██████████|

Downloading: 100%|██████████|
Downloading: 100%|██████████|


In [20]:
aggregate_reports = pd.concat([results[i][0] for i in range(len(results))]).sort_values(by='Report_Month')
acc_type_reports = pd.concat([results[i][1] for i in range(len(results))]).sort_values(by=['Report_Month', 'Account_Type'])

with pd.ExcelWriter(f'push_channel_report_quarterly.xlsx') as writer:
    aggregate_reports.to_excel(writer, sheet_name='total')
    acc_type_reports.to_excel(writer, sheet_name='acc_type')